In [1]:
#importing required libraries
import os
import shutil
import time

import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

from glob import glob
import numpy as np

import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
#reading 2013 data into df1
df1 = pd.read_csv('../Data/2013/Divvy_Trips_2013.csv')

D:\Softwares_Installed\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#printing the head of dataframe
df1.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthday
0,4118,2013-06-27 12:11,2013-06-27 12:16,480,316,85,Michigan Ave & Oak St,28,Larrabee St & Menomonee St,Customer,NaN,NaN
1,4275,2013-06-27 14:44,2013-06-27 14:45,77,64,32,Racine Ave & Congress Pkwy,32,Racine Ave & Congress Pkwy,Customer,NaN,NaN
2,4291,2013-06-27 14:58,2013-06-27 15:05,77,433,32,Racine Ave & Congress Pkwy,19,Loomis St & Taylor St,Customer,NaN,NaN
3,4316,2013-06-27 15:06,2013-06-27 15:09,77,123,19,Loomis St & Taylor St,19,Loomis St & Taylor St,Customer,NaN,NaN
4,4342,2013-06-27 15:13,2013-06-27 15:27,77,852,19,Loomis St & Taylor St,55,Halsted St & James M Rochford St,Customer,NaN,NaN


In [4]:
#checking all column names of the dataframe
df1.columns

Index(['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthday'],
      dtype='object')

### Saving all the station names in a list

#### Since divvy was establised in 2013, all the stations that are in this year's dataset will be added to the map

In [5]:
#saving all the stations that have unique names in a list
allSt13  = list (df1['from_station_name'].unique())

In [6]:
#length of the list (Total number of stations in 2013)
len(allSt13)

300

# Plotting all added stations of 2013

In [7]:
#reading the 'final_stations.csv' into a dataframe
#using all the stations co-ordinates and comparing with each of the year's stations added/ removed list and adding locations 
#     to each of the list to plot yearly data.

final = pd.read_csv('final_stations.csv')

In [8]:
final.head()

,Unnamed: 0,Station,Latitude,Longitude
0,0,Ogden Ave & Race Ave,41.891795,-87.658751
1,1,State St & 35th St,41.831036,-87.626798
2,2,Clark St & Elm St,41.902973,-87.631280
3,3,Carpenter St & 63rd St,41.779870,-87.650919
4,4,Leavitt St & Division St,41.902997,-87.683825


In [9]:
#Removing the unnecessary column in the dataframe
final = final.drop('Unnamed: 0', axis =1)

In [10]:
final.head()

,Station,Latitude,Longitude
0,Ogden Ave & Race Ave,41.891795,-87.658751
1,State St & 35th St,41.831036,-87.626798
2,Clark St & Elm St,41.902973,-87.631280
3,Carpenter St & 63rd St,41.779870,-87.650919
4,Leavitt St & Division St,41.902997,-87.683825


In [11]:
final[final['Station'] == 'Emerald Ave & 31st St']

,Station,Latitude,Longitude
727,Emerald Ave & 31st St,41.838198,-87.645143


In [12]:
#Converting the dataframe into a dictionary
# key : Station name and value: latitude and longitude

locations = dict()

for station in final['Station']:
    locations[station] = {'lat': final[final['Station'] == station]['Latitude'].to_list()[0], 'long': final[final['Station'] == station]['Longitude'].to_list()[0]}

#printing out the first item in the dictionary to check the key-value pair
list(locations.items())[0]

('Ogden Ave & Race Ave', {'lat': 41.891795, 'long': -87.658751})

In [13]:
#adding co-ordinates for the stations in allSt13
n13 = {k: locations[k] for k in locations.keys() & set(allSt13)}

#printing out the first item in the dictionary to check the key-value pair
list(n13.items())[0]

('Michigan Ave & 14th St', {'lat': 41.86405900000001, 'long': -87.623727})

In [14]:
#Converting the dictionary into dataframe
st13 = pd.DataFrame.from_dict(n13, orient='index')

In [15]:
#Resetting the index
st13.reset_index(inplace=True)

#Renaming the index column to Station name
st13 = st13.rename(columns = {'index':'Station Name'})

In [16]:
#Adding a column with values of color for every station
st13['color'] = st13.apply(lambda row: 'blue', axis=1)

#printing to check the structure of the dataframe
st13.head(1)

,Station Name,lat,long,color
0,Michigan Ave & 14th St,41.864059,-87.623727,blue


# Location Marker map for interactivity purpose (Station Name)

In [17]:
#setting marker as the color mentioned in the color column of the dataframe
marker = st13.loc[st13['color']!='']

map = folium.Map(location=[marker.lat.mean(), marker.long.mean()], zoom_start=11,
                     width=980,height=590, control_scale=True, tiles='OpenStreetMap')
marker.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]],
        popup=row['Station Name'],
          icon=folium.Icon(color=row['color'])
            ).add_to(map), axis=1)

map

# Circle marker map 2013
## Saving the map for converting into gif

In [18]:
#create a map
m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='cartodb positron')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2013')   
m.get_root().html.add_child(folium.Element(title_html))

#creates a CircleMarker and adds it to the map
def plotDot(point):
    folium.CircleMarker(location=[point.lat, point.long],             
                        color= "green",
                        radius=3,
                        weight=2
                       ).add_to(m)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
st13.apply(plotDot, axis = 1)

m

In [19]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [20]:
m.save('Gif_circlemarker/2013.html')

In [21]:
with fileinput.FileInput('Gif_circlemarker/2013.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [24]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_circlemarker')

fn='2013.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2013.png')
browser.quit()

# remove html files

for i in range (3,10,1):
    if os.path.exists('201' + str(i) + '.html'):
        os.remove('201' + str(i) + '.html')
    
for i in range(0,3,1):
    if os.path.exists('202' + str(i) + '.html'):
        os.remove('202' + str(i) + '.html')    

In [25]:
image = Image.open('ss2013.png')
box = (0, 0, 1100, 700)
cropped_image = image.crop(box)
cropped_image.save('croppedss2013.png')

for i in range (3,10,1):
    if os.path.exists('ss201' + str(i) + '.png'):
        os.remove('ss201' + str(i) + '.png')
    
for i in range(0,3,1):
    if os.path.exists('ss202' + str(i) + '.png'):
        os.remove('ss202' + str(i) + '.png')    

# Heatmap 2013
## Saving the map for converting into gif

In [ ]:
numlist = st13.reset_index()[['lat', 'long']].values.tolist()


m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='OpenStreetMap')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2013')   
m.get_root().html.add_child(folium.Element(title_html))


HeatMap(numlist).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m

In [ ]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [ ]:
m.save('Gif_heatmap/2013.html')

In [ ]:
with fileinput.FileInput('Gif_heatmap/2013.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [ ]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_heatmap')

fn='2013.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2013.png')
browser.quit()

# remove html files
for i in range (3,9,1):
    os.remove('201' + str(i) + '.html')
    
for i in range(0,2,1):
    os.remove('202' + str(i) + '.html')    

In [ ]:
image = Image.open('ss2013.png')
box = (0, 0, 1250, 780)
cropped_image = image.crop(box)
cropped_image.save('croppedss2013.png')

# Creating png for heatmap


In [ ]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_heatmap')

In [ ]:
image_path = Path()

images = list(image_path.glob('croppedss*.png'))
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))
    os.remove(file_name)
    
imageio.mimwrite('Spread of Divvy stations across Years.gif', image_list, fps=1)

# Creating png for circle marker


In [ ]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_circlemarker')

In [ ]:
image_path = Path()

images = list(image_path.glob('croppedss*.png'))
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))
    os.remove(file_name)
    
imageio.mimwrite('GifMap.gif', image_list, fps=1)